In [2]:
import sys
import cython
import numpy as np
import scipy.sparse as sps

sys.path.append('../RecSysRep/')

In [3]:
import Basics.Load as ld

URM_all, ICM_genre_all, ICM_subgenre_all, ICM_channel_all, ICM_event_all = ld.getCOOs()
# URM_train, URM_val = ld.getSplit(URM_train_val, 5678, 0.8)

In [4]:
import os

ICM_all = sps.hstack([ICM_genre_all, ICM_subgenre_all, ICM_channel_all])

output_folder_path = "result_experiments/S-SLIM_BPR/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 10  # using 10 as an example
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

# SLIM Model

In [5]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

#URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)

In [6]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
#evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13643 ( 0.1%) Users that have less than 1 test interactions


In [28]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {
    "epochs": Categorical([2000]),
    "lambda_i": Real(low = 1e-5, high = 1e-2, prior = 'log-uniform'),
    "lambda_j": Real(low = 1e-5, high = 1e-2, prior = 'log-uniform'),
    "learning_rate": Real(low = 0.5e-2, high = 1e-1, prior = 'log-uniform'),
    "topK": Integer(100, 1000),
    "random_seed":Categorical([1234]),
    "sgd_mode": Categorical(["sgd", "adagrad", "adam"])
}

earlystopping_keywargs = {"validation_every_n": 10,
                          "stop_on_validation": True,
                          "evaluator_object": evaluator_validation,
                          "lower_validations_allowed": 10,
                          "validation_metric": metric_to_optimize,
                          }

In [8]:
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_S_BPR_Cython
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = SLIM_S_BPR_Cython

hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                         evaluator_validation=evaluator_validation)

In [30]:
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train, ICM_all],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = earlystopping_keywargs     # Additiona hyperparameters for the fit function
)

In [ ]:
hyperparameterSearch.search(recommender_input_args,
                       hyperparameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       output_folder_path = output_folder_path, # Where to save the results
                       output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                       metric_to_optimize = metric_to_optimize,
                       cutoff_to_optimize = cutoff_to_optimize,
                      )

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'epochs': 2000, 'mw': 0.8007909783324567, 'lambda_i': 0.004102787540371504, 'lambda_j': 5.577012634469088e-05, 'learning_rate': 0.0007912384382146588, 'topK': 948, 'random_seed': 1234, 'sgd_mode': 'adagrad'}
SLIM_S_BPR_Recommender: Automatic selection of fastest train mode. Available RAM is 9796.00 MB (63.75%) of 15366.00 MB, required is 1304.51 MB. Using dense matrix.
Processed 13658 (100.0%) in 0.76 sec. BPR loss is 2.28E-02. Sample per second: 17863
SLIM_S_BPR_Recommender: Epoch 1 of 2000. Elapsed time 0.37 sec
Processed 13658 (100.0%) in 1.14 sec. BPR loss is 8.01E-02. Sample per second: 11955
SLIM_S_BPR_Recommender: Epoch 2 of 2000. Elapsed time 0.75 sec
Processed 13658 (100.0%) in 0.52 sec. BPR loss is 1.39E-01. Sample per second: 26331
SLIM_S_BPR_Recommender: Epoch 3 of 2000. Elapsed time 1.12 sec
Processed 13658 (100.0%) in 0.90 sec. BPR loss is 2.03E-01. Sample per second: 15221

Processed 13658 (100.0%) in 1.37 sec. BPR loss is 1.19E+00. Sample per second: 10003
SLIM_S_BPR_Recommender: Epoch 26 of 2000. Elapsed time 2.38 min
Processed 13658 (100.0%) in 0.75 sec. BPR loss is 1.23E+00. Sample per second: 18295
SLIM_S_BPR_Recommender: Epoch 27 of 2000. Elapsed time 2.39 min
Processed 13658 (100.0%) in 1.13 sec. BPR loss is 1.27E+00. Sample per second: 12110
SLIM_S_BPR_Recommender: Epoch 28 of 2000. Elapsed time 2.40 min
Processed 13658 (100.0%) in 0.50 sec. BPR loss is 1.28E+00. Sample per second: 27130
SLIM_S_BPR_Recommender: Epoch 29 of 2000. Elapsed time 2.40 min
Processed 13658 (100.0%) in 0.89 sec. BPR loss is 1.31E+00. Sample per second: 15406
SLIM_S_BPR_Recommender: Validation begins...
EvaluatorHoldout: Processed 13639 (100.0%) in 16.69 sec. Users per second: 817
SLIM_S_BPR_Recommender: CUTOFF: 10 - PRECISION: 0.3133001, PRECISION_RECALL_MIN_DEN: 0.3145643, RECALL: 0.0521269, MAP: 0.1892596, MAP_MIN_DEN: 0.1899381, MRR: 0.6049270, NDCG: 0.3379749, F1: 0.0

Processed 13658 (100.0%) in 1.21 sec. BPR loss is 1.96E+00. Sample per second: 11324
SLIM_S_BPR_Recommender: Epoch 51 of 2000. Elapsed time 4.86 min
Processed 13658 (100.0%) in 0.58 sec. BPR loss is 1.91E+00. Sample per second: 23451
SLIM_S_BPR_Recommender: Epoch 52 of 2000. Elapsed time 4.87 min
Processed 13658 (100.0%) in 0.95 sec. BPR loss is 1.99E+00. Sample per second: 14361
SLIM_S_BPR_Recommender: Epoch 53 of 2000. Elapsed time 4.88 min
Processed 13658 (100.0%) in 1.33 sec. BPR loss is 1.96E+00. Sample per second: 10266
SLIM_S_BPR_Recommender: Epoch 54 of 2000. Elapsed time 4.88 min
Processed 13658 (100.0%) in 0.70 sec. BPR loss is 1.99E+00. Sample per second: 19623
SLIM_S_BPR_Recommender: Validation begins...
EvaluatorHoldout: Processed 13639 (100.0%) in 16.67 sec. Users per second: 818
SLIM_S_BPR_Recommender: CUTOFF: 10 - PRECISION: 0.3152577, PRECISION_RECALL_MIN_DEN: 0.3165414, RECALL: 0.0525443, MAP: 0.1906393, MAP_MIN_DEN: 0.1913258, MRR: 0.6070001, NDCG: 0.3398899, F1: 0.0

Processed 13658 (100.0%) in 0.91 sec. BPR loss is 2.58E+00. Sample per second: 15033
SLIM_S_BPR_Recommender: Epoch 76 of 2000. Elapsed time 7.38 min
Processed 13658 (100.0%) in 1.29 sec. BPR loss is 2.56E+00. Sample per second: 10621
SLIM_S_BPR_Recommender: Epoch 77 of 2000. Elapsed time 7.38 min
Processed 13658 (100.0%) in 0.66 sec. BPR loss is 2.58E+00. Sample per second: 20543
SLIM_S_BPR_Recommender: Epoch 78 of 2000. Elapsed time 7.39 min
Processed 13658 (100.0%) in 1.04 sec. BPR loss is 2.64E+00. Sample per second: 13086
SLIM_S_BPR_Recommender: Epoch 79 of 2000. Elapsed time 7.39 min
Processed 13658 (100.0%) in 0.42 sec. BPR loss is 2.59E+00. Sample per second: 32512
SLIM_S_BPR_Recommender: Validation begins...
EvaluatorHoldout: Processed 13639 (100.0%) in 16.47 sec. Users per second: 828
SLIM_S_BPR_Recommender: CUTOFF: 10 - PRECISION: 0.3168267, PRECISION_RECALL_MIN_DEN: 0.3181611, RECALL: 0.0529565, MAP: 0.1916355, MAP_MIN_DEN: 0.1923505, MRR: 0.6081003, NDCG: 0.3413397, F1: 0.0

Processed 13658 (100.0%) in 1.18 sec. BPR loss is 2.96E+00. Sample per second: 11617
SLIM_S_BPR_Recommender: Epoch 101 of 2000. Elapsed time 9.90 min
Processed 13658 (100.0%) in 0.55 sec. BPR loss is 3.01E+00. Sample per second: 24835
SLIM_S_BPR_Recommender: Epoch 102 of 2000. Elapsed time 9.90 min
Processed 13658 (100.0%) in 0.92 sec. BPR loss is 3.12E+00. Sample per second: 14805
SLIM_S_BPR_Recommender: Epoch 103 of 2000. Elapsed time 9.91 min
Processed 13658 (100.0%) in 1.29 sec. BPR loss is 3.04E+00. Sample per second: 10548
SLIM_S_BPR_Recommender: Epoch 104 of 2000. Elapsed time 9.91 min
Processed 13658 (100.0%) in 0.66 sec. BPR loss is 3.07E+00. Sample per second: 20593
SLIM_S_BPR_Recommender: Validation begins...
EvaluatorHoldout: Processed 13639 (100.0%) in 16.45 sec. Users per second: 829
SLIM_S_BPR_Recommender: CUTOFF: 10 - PRECISION: 0.3183225, PRECISION_RECALL_MIN_DEN: 0.3196327, RECALL: 0.0532345, MAP: 0.1925179, MAP_MIN_DEN: 0.1932104, MRR: 0.6088082, NDCG: 0.3425953, F1:

...

Processed 13658 (100.0%) in 0.76 sec. BPR loss is 1.35E+01. Sample per second: 17959
SLIM_S_BPR_Recommender: Epoch 1481 of 2000. Elapsed time 2.36 hour
Processed 13658 (100.0%) in 1.05 sec. BPR loss is 1.33E+01. Sample per second: 13041
SLIM_S_BPR_Recommender: Epoch 1482 of 2000. Elapsed time 2.36 hour
Processed 13658 (100.0%) in 0.33 sec. BPR loss is 1.37E+01. Sample per second: 40802
SLIM_S_BPR_Recommender: Epoch 1483 of 2000. Elapsed time 2.36 hour
Processed 13658 (100.0%) in 0.62 sec. BPR loss is 1.36E+01. Sample per second: 22029
SLIM_S_BPR_Recommender: Epoch 1484 of 2000. Elapsed time 2.36 hour
Processed 13658 (100.0%) in 0.91 sec. BPR loss is 1.39E+01. Sample per second: 15026
SLIM_S_BPR_Recommender: Validation begins...
EvaluatorHoldout: Processed 13639 (100.0%) in 16.62 sec. Users per second: 821
SLIM_S_BPR_Recommender: CUTOFF: 10 - PRECISION: 0.3388078, PRECISION_RECALL_MIN_DEN: 0.3403785, RECALL: 0.0584296, MAP: 0.2043621, MAP_MIN_DEN: 0.2052322, MRR: 0.6193927, NDCG: 0.3600

SLIM_S_BPR_Recommender: Epoch 1506 of 2000. Elapsed time 2.40 hour
Processed 13658 (100.0%) in 0.36 sec. BPR loss is 1.42E+01. Sample per second: 38200
SLIM_S_BPR_Recommender: Epoch 1507 of 2000. Elapsed time 2.40 hour
Processed 13658 (100.0%) in 0.64 sec. BPR loss is 1.36E+01. Sample per second: 21218
SLIM_S_BPR_Recommender: Epoch 1508 of 2000. Elapsed time 2.40 hour
Processed 13658 (100.0%) in 0.93 sec. BPR loss is 1.39E+01. Sample per second: 14705
SLIM_S_BPR_Recommender: Epoch 1509 of 2000. Elapsed time 2.40 hour
Processed 13658 (100.0%) in 1.22 sec. BPR loss is 1.39E+01. Sample per second: 11212
SLIM_S_BPR_Recommender: Validation begins...
EvaluatorHoldout: Processed 13639 (100.0%) in 16.63 sec. Users per second: 820
SLIM_S_BPR_Recommender: CUTOFF: 10 - PRECISION: 0.3391598, PRECISION_RECALL_MIN_DEN: 0.3407304, RECALL: 0.0585218, MAP: 0.2045339, MAP_MIN_DEN: 0.2054053, MRR: 0.6192499, NDCG: 0.3603186, F1: 0.0998197, HIT_RATE: 0.9517560, ARHR_ALL_HITS: 1.1071577, NOVELTY: 0.0052121

SLIM_S_BPR_Recommender: Epoch 1532 of 2000. Elapsed time 2.44 hour
Processed 13658 (100.0%) in 0.95 sec. BPR loss is 1.41E+01. Sample per second: 14385
SLIM_S_BPR_Recommender: Epoch 1533 of 2000. Elapsed time 2.44 hour
Processed 13658 (100.0%) in 1.24 sec. BPR loss is 1.40E+01. Sample per second: 11016
SLIM_S_BPR_Recommender: Epoch 1534 of 2000. Elapsed time 2.44 hour
Processed 13658 (100.0%) in 0.53 sec. BPR loss is 1.39E+01. Sample per second: 25808
SLIM_S_BPR_Recommender: Validation begins...
EvaluatorHoldout: Processed 13639 (100.0%) in 16.46 sec. Users per second: 829
SLIM_S_BPR_Recommender: CUTOFF: 10 - PRECISION: 0.3391891, PRECISION_RECALL_MIN_DEN: 0.3407597, RECALL: 0.0585103, MAP: 0.2047406, MAP_MIN_DEN: 0.2056154, MRR: 0.6197864, NDCG: 0.3605215, F1: 0.0998043, HIT_RATE: 0.9519026, ARHR_ALL_HITS: 1.1081819, NOVELTY: 0.0052128, AVERAGE_POPULARITY: 0.7137852, DIVERSITY_MEAN_INTER_LIST: 0.7163259, DIVERSITY_HERFINDAHL: 0.9716273, COVERAGE_ITEM: 0.0188272, COVERAGE_ITEM_CORRECT:

Processed 13658 (100.0%) in 1.23 sec. BPR loss is 1.40E+01. Sample per second: 11116
SLIM_S_BPR_Recommender: Epoch 1558 of 2000. Elapsed time 2.48 hour
Processed 13658 (100.0%) in 0.52 sec. BPR loss is 1.39E+01. Sample per second: 26317
SLIM_S_BPR_Recommender: Epoch 1559 of 2000. Elapsed time 2.48 hour
Processed 13658 (100.0%) in 0.81 sec. BPR loss is 1.35E+01. Sample per second: 16871
SLIM_S_BPR_Recommender: Validation begins...
EvaluatorHoldout: Processed 13639 (100.0%) in 16.48 sec. Users per second: 828
SLIM_S_BPR_Recommender: CUTOFF: 10 - PRECISION: 0.3394310, PRECISION_RECALL_MIN_DEN: 0.3410098, RECALL: 0.0585687, MAP: 0.2048272, MAP_MIN_DEN: 0.2057054, MRR: 0.6198663, NDCG: 0.3606842, F1: 0.0998998, HIT_RATE: 0.9521226, ARHR_ALL_HITS: 1.1084050, NOVELTY: 0.0052135, AVERAGE_POPULARITY: 0.7133962, DIVERSITY_MEAN_INTER_LIST: 0.7168862, DIVERSITY_HERFINDAHL: 0.9716834, COVERAGE_ITEM: 0.0190487, COVERAGE_ITEM_CORRECT: 0.0118500, COVERAGE_USER: 0.9991941, COVERAGE_USER_CORRECT: 0.9513

SLIM_S_BPR_Recommender: Epoch 1583 of 2000. Elapsed time 2.52 hour
Processed 13658 (100.0%) in 1.23 sec. BPR loss is 1.38E+01. Sample per second: 11127
SLIM_S_BPR_Recommender: Epoch 1584 of 2000. Elapsed time 2.52 hour
Processed 13658 (100.0%) in 0.52 sec. BPR loss is 1.40E+01. Sample per second: 26410
SLIM_S_BPR_Recommender: Validation begins...
EvaluatorHoldout: Processed 13639 (100.0%) in 16.61 sec. Users per second: 821
SLIM_S_BPR_Recommender: CUTOFF: 10 - PRECISION: 0.3395850, PRECISION_RECALL_MIN_DEN: 0.3411534, RECALL: 0.0586124, MAP: 0.2049909, MAP_MIN_DEN: 0.2058642, MRR: 0.6199609, NDCG: 0.3608794, F1: 0.0999700, HIT_RATE: 0.9525625, ARHR_ALL_HITS: 1.1090702, NOVELTY: 0.0052145, AVERAGE_POPULARITY: 0.7128371, DIVERSITY_MEAN_INTER_LIST: 0.7179739, DIVERSITY_HERFINDAHL: 0.9717921, COVERAGE_ITEM: 0.0189933, COVERAGE_ITEM_CORRECT: 0.0119054, COVERAGE_USER: 0.9991941, COVERAGE_USER_CORRECT: 0.9517949, DIVERSITY_GINI: 0.0020362, SHANNON_ENTROPY: 5.5692053, RATIO_DIVERSITY_HERFINDAH

In [9]:
from Recommenders.DataIO import DataIO

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")

search_metadata.keys()

dict_keys(['exception_list', 'time_on_train_total', 'hyperparameters_df', 'time_df', 'algorithm_name_recommender', 'metric_to_optimize', 'algorithm_name_search', 'result_on_last', 'result_on_test_best', 'hyperparameters_best_index', 'cutoff_to_optimize', 'time_on_validation_total', 'result_on_validation_best', 'result_on_test_df', 'hyperparameters_best', 'time_on_last_df', 'time_on_validation_avg', 'time_on_test_avg', 'time_on_test_total', 'result_on_validation_df', 'time_on_train_avg'])

In [10]:
hyp = search_metadata["hyperparameters_best"]
hyp

{'epochs': 750,
 'lambda_i': 1.3263293830505987e-05,
 'lambda_j': 2.4688952316384904e-05,
 'learning_rate': 0.0048039449572740995,
 'topK': 524,
 'random_seed': 1234,
 'sgd_mode': 'sgd'}

In [ ]:
result_on_validation_df = search_metadata["result_on_test_df"]
result_on_validation_df

In [ ]:
'''
from Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_FunkSVD_Cython
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_object = MatrixFactorization_FunkSVD_Cython(URM_all.tocsr())

recommender_object.load_model(output_folder_path, 
                              file_name = recommender_object.RECOMMENDER_NAME + "_best_model_last.zip" )
'''

In [14]:
recommender = SLIM_S_BPR_Cython(URM_all.tocsr(), ICM_all)
K = 10

recommender.fit(epochs=hyp["epochs"], lambda_i=hyp["lambda_i"], lambda_j=hyp["lambda_j"], 
                learning_rate=hyp["learning_rate"], topK=hyp["topK"], sgd_mode=hyp["sgd_mode"], random_seed=hyp['random_seed'])




Deallocating Cython objects
SLIM_S_BPR_Recommender: Automatic selection of fastest train mode. Available RAM is 8685.00 MB (56.52%) of 15366.00 MB, required is 1304.51 MB. Using dense matrix.
Processed 13984 (100.0%) in 1.64 sec. BPR loss is 7.15E-01. Sample per second: 8518
SLIM_S_BPR_Recommender: Epoch 1 of 750. Elapsed time 0.88 sec
Processed 13984 (100.0%) in 1.54 sec. BPR loss is 2.40E+00. Sample per second: 9058
SLIM_S_BPR_Recommender: Epoch 2 of 750. Elapsed time 1.78 sec
Processed 13984 (100.0%) in 1.47 sec. BPR loss is 3.91E+00. Sample per second: 9538
SLIM_S_BPR_Recommender: Epoch 3 of 750. Elapsed time 2.71 sec
Processed 13984 (100.0%) in 1.42 sec. BPR loss is 5.12E+00. Sample per second: 9862
SLIM_S_BPR_Recommender: Epoch 4 of 750. Elapsed time 3.66 sec
Processed 13984 (100.0%) in 1.41 sec. BPR loss is 6.25E+00. Sample per second: 9890
SLIM_S_BPR_Recommender: Epoch 5 of 750. Elapsed time 4.65 sec
Processed 13984 (100.0%) in 1.45 sec. BPR loss is 7.46E+00. Sample per second:

SLIM_S_BPR_Recommender: Epoch 55 of 750. Elapsed time 1.36 min
Processed 13984 (100.0%) in 2.05 sec. BPR loss is 2.17E+01. Sample per second: 6829
SLIM_S_BPR_Recommender: Epoch 56 of 750. Elapsed time 1.39 min
Processed 13984 (100.0%) in 1.81 sec. BPR loss is 2.20E+01. Sample per second: 7721
SLIM_S_BPR_Recommender: Epoch 57 of 750. Elapsed time 1.42 min
Processed 13984 (100.0%) in 2.59 sec. BPR loss is 2.20E+01. Sample per second: 5398
SLIM_S_BPR_Recommender: Epoch 58 of 750. Elapsed time 1.45 min
Processed 13984 (100.0%) in 2.35 sec. BPR loss is 2.22E+01. Sample per second: 5956
SLIM_S_BPR_Recommender: Epoch 59 of 750. Elapsed time 1.48 min
Processed 13984 (100.0%) in 2.09 sec. BPR loss is 2.28E+01. Sample per second: 6699
SLIM_S_BPR_Recommender: Epoch 60 of 750. Elapsed time 1.51 min
Processed 13984 (100.0%) in 1.81 sec. BPR loss is 2.19E+01. Sample per second: 7732
SLIM_S_BPR_Recommender: Epoch 61 of 750. Elapsed time 1.53 min
Processed 13984 (100.0%) in 2.56 sec. BPR loss is 2.20E

SLIM_S_BPR_Recommender: Epoch 111 of 750. Elapsed time 2.96 min
Processed 13984 (100.0%) in 2.14 sec. BPR loss is 2.62E+01. Sample per second: 6519
SLIM_S_BPR_Recommender: Epoch 112 of 750. Elapsed time 2.99 min
Processed 13984 (100.0%) in 1.89 sec. BPR loss is 2.69E+01. Sample per second: 7411
SLIM_S_BPR_Recommender: Epoch 113 of 750. Elapsed time 3.02 min
Processed 13984 (100.0%) in 2.62 sec. BPR loss is 2.66E+01. Sample per second: 5341
SLIM_S_BPR_Recommender: Epoch 114 of 750. Elapsed time 3.05 min
Processed 13984 (100.0%) in 2.38 sec. BPR loss is 2.66E+01. Sample per second: 5868
SLIM_S_BPR_Recommender: Epoch 115 of 750. Elapsed time 3.08 min
Processed 13984 (100.0%) in 2.16 sec. BPR loss is 2.72E+01. Sample per second: 6462
SLIM_S_BPR_Recommender: Epoch 116 of 750. Elapsed time 3.11 min
Processed 13984 (100.0%) in 1.89 sec. BPR loss is 2.69E+01. Sample per second: 7396
SLIM_S_BPR_Recommender: Epoch 117 of 750. Elapsed time 3.14 min
Processed 13984 (100.0%) in 2.65 sec. BPR loss i

Processed 13984 (100.0%) in 1.85 sec. BPR loss is 2.91E+01. Sample per second: 7539
SLIM_S_BPR_Recommender: Epoch 167 of 750. Elapsed time 4.58 min
Processed 13984 (100.0%) in 1.73 sec. BPR loss is 2.92E+01. Sample per second: 8079
SLIM_S_BPR_Recommender: Epoch 168 of 750. Elapsed time 4.60 min
Processed 13984 (100.0%) in 1.61 sec. BPR loss is 2.93E+01. Sample per second: 8696
SLIM_S_BPR_Recommender: Epoch 169 of 750. Elapsed time 4.61 min
Processed 13984 (100.0%) in 1.48 sec. BPR loss is 2.92E+01. Sample per second: 9417
SLIM_S_BPR_Recommender: Epoch 170 of 750. Elapsed time 4.63 min
Processed 13984 (100.0%) in 1.37 sec. BPR loss is 2.84E+01. Sample per second: 10219
SLIM_S_BPR_Recommender: Epoch 171 of 750. Elapsed time 4.64 min
Processed 13984 (100.0%) in 1.25 sec. BPR loss is 3.03E+01. Sample per second: 11180
SLIM_S_BPR_Recommender: Epoch 172 of 750. Elapsed time 4.66 min
Processed 13984 (100.0%) in 1.13 sec. BPR loss is 2.89E+01. Sample per second: 12395
SLIM_S_BPR_Recommender: E

SLIM_S_BPR_Recommender: Epoch 222 of 750. Elapsed time 5.69 min
Processed 13984 (100.0%) in 1.94 sec. BPR loss is 3.00E+01. Sample per second: 7219
SLIM_S_BPR_Recommender: Epoch 223 of 750. Elapsed time 5.72 min
Processed 13984 (100.0%) in 2.60 sec. BPR loss is 3.11E+01. Sample per second: 5378
SLIM_S_BPR_Recommender: Epoch 224 of 750. Elapsed time 5.75 min
Processed 13984 (100.0%) in 2.25 sec. BPR loss is 3.16E+01. Sample per second: 6207
SLIM_S_BPR_Recommender: Epoch 225 of 750. Elapsed time 5.77 min
Processed 13984 (100.0%) in 1.84 sec. BPR loss is 3.12E+01. Sample per second: 7610
SLIM_S_BPR_Recommender: Epoch 226 of 750. Elapsed time 5.80 min
Processed 13984 (100.0%) in 1.69 sec. BPR loss is 2.96E+01. Sample per second: 8281
SLIM_S_BPR_Recommender: Epoch 227 of 750. Elapsed time 5.82 min
Processed 13984 (100.0%) in 1.56 sec. BPR loss is 3.08E+01. Sample per second: 8981
SLIM_S_BPR_Recommender: Epoch 228 of 750. Elapsed time 5.83 min
Processed 13984 (100.0%) in 1.39 sec. BPR loss i

...

Processed 13984 (100.0%) in 1.54 sec. BPR loss is 3.68E+01. Sample per second: 9068
SLIM_S_BPR_Recommender: Epoch 500 of 750. Elapsed time 9.93 min
Processed 13984 (100.0%) in 1.38 sec. BPR loss is 3.65E+01. Sample per second: 10117
SLIM_S_BPR_Recommender: Epoch 501 of 750. Elapsed time 9.94 min
Processed 13984 (100.0%) in 1.24 sec. BPR loss is 3.71E+01. Sample per second: 11316
SLIM_S_BPR_Recommender: Epoch 502 of 750. Elapsed time 9.96 min
Processed 13984 (100.0%) in 1.08 sec. BPR loss is 3.82E+01. Sample per second: 12887
SLIM_S_BPR_Recommender: Epoch 503 of 750. Elapsed time 9.97 min
Processed 13984 (100.0%) in 0.93 sec. BPR loss is 3.63E+01. Sample per second: 15084
SLIM_S_BPR_Recommender: Epoch 504 of 750. Elapsed time 9.99 min
Processed 13984 (100.0%) in 1.78 sec. BPR loss is 3.74E+01. Sample per second: 7869
SLIM_S_BPR_Recommender: Epoch 505 of 750. Elapsed time 10.00 min
Processed 13984 (100.0%) in 1.61 sec. BPR loss is 3.54E+01. Sample per second: 8694
SLIM_S_BPR_Recommender:

Processed 13984 (100.0%) in 1.74 sec. BPR loss is 3.61E+01. Sample per second: 8020
SLIM_S_BPR_Recommender: Epoch 555 of 750. Elapsed time 11.07 min
Processed 13984 (100.0%) in 2.40 sec. BPR loss is 3.74E+01. Sample per second: 5817
SLIM_S_BPR_Recommender: Epoch 556 of 750. Elapsed time 11.09 min
Processed 13984 (100.0%) in 2.10 sec. BPR loss is 3.76E+01. Sample per second: 6662
SLIM_S_BPR_Recommender: Epoch 557 of 750. Elapsed time 11.12 min
Processed 13984 (100.0%) in 1.75 sec. BPR loss is 3.71E+01. Sample per second: 7972
SLIM_S_BPR_Recommender: Epoch 558 of 750. Elapsed time 11.15 min
Processed 13984 (100.0%) in 2.43 sec. BPR loss is 3.73E+01. Sample per second: 5753
SLIM_S_BPR_Recommender: Epoch 559 of 750. Elapsed time 11.18 min
Processed 13984 (100.0%) in 2.08 sec. BPR loss is 4.02E+01. Sample per second: 6709
SLIM_S_BPR_Recommender: Epoch 560 of 750. Elapsed time 11.21 min
Processed 13984 (100.0%) in 1.76 sec. BPR loss is 3.69E+01. Sample per second: 7958
SLIM_S_BPR_Recommender

Processed 13984 (100.0%) in 2.30 sec. BPR loss is 3.71E+01. Sample per second: 6084
SLIM_S_BPR_Recommender: Epoch 610 of 750. Elapsed time 12.59 min
Processed 13984 (100.0%) in 1.97 sec. BPR loss is 4.15E+01. Sample per second: 7083
SLIM_S_BPR_Recommender: Epoch 611 of 750. Elapsed time 12.62 min
Processed 13984 (100.0%) in 2.65 sec. BPR loss is 3.87E+01. Sample per second: 5281
SLIM_S_BPR_Recommender: Epoch 612 of 750. Elapsed time 12.65 min
Processed 13984 (100.0%) in 2.30 sec. BPR loss is 3.81E+01. Sample per second: 6067
SLIM_S_BPR_Recommender: Epoch 613 of 750. Elapsed time 12.68 min
Processed 13984 (100.0%) in 1.97 sec. BPR loss is 3.61E+01. Sample per second: 7108
SLIM_S_BPR_Recommender: Epoch 614 of 750. Elapsed time 12.70 min
Processed 13984 (100.0%) in 2.63 sec. BPR loss is 3.79E+01. Sample per second: 5307
SLIM_S_BPR_Recommender: Epoch 615 of 750. Elapsed time 12.73 min
Processed 13984 (100.0%) in 2.29 sec. BPR loss is 3.84E+01. Sample per second: 6104
SLIM_S_BPR_Recommender

Processed 13984 (100.0%) in 1.41 sec. BPR loss is 3.91E+01. Sample per second: 9907
SLIM_S_BPR_Recommender: Epoch 665 of 750. Elapsed time 13.88 min
Processed 13984 (100.0%) in 1.23 sec. BPR loss is 4.04E+01. Sample per second: 11367
SLIM_S_BPR_Recommender: Epoch 666 of 750. Elapsed time 13.89 min
Processed 13984 (100.0%) in 1.09 sec. BPR loss is 3.97E+01. Sample per second: 12856
SLIM_S_BPR_Recommender: Epoch 667 of 750. Elapsed time 13.91 min
Processed 13984 (100.0%) in 0.93 sec. BPR loss is 4.01E+01. Sample per second: 15094
SLIM_S_BPR_Recommender: Epoch 668 of 750. Elapsed time 13.92 min
Processed 13984 (100.0%) in 1.82 sec. BPR loss is 3.85E+01. Sample per second: 7696
SLIM_S_BPR_Recommender: Epoch 669 of 750. Elapsed time 13.93 min
Processed 13984 (100.0%) in 1.69 sec. BPR loss is 3.57E+01. Sample per second: 8296
SLIM_S_BPR_Recommender: Epoch 670 of 750. Elapsed time 13.95 min
Processed 13984 (100.0%) in 1.59 sec. BPR loss is 3.88E+01. Sample per second: 8816
SLIM_S_BPR_Recommen

Processed 13984 (100.0%) in 1.88 sec. BPR loss is 3.96E+01. Sample per second: 7426
SLIM_S_BPR_Recommender: Epoch 720 of 750. Elapsed time 15.04 min
Processed 13984 (100.0%) in 2.56 sec. BPR loss is 4.06E+01. Sample per second: 5467
SLIM_S_BPR_Recommender: Epoch 721 of 750. Elapsed time 15.06 min
Processed 13984 (100.0%) in 2.24 sec. BPR loss is 4.15E+01. Sample per second: 6254
SLIM_S_BPR_Recommender: Epoch 722 of 750. Elapsed time 15.09 min
Processed 13984 (100.0%) in 1.87 sec. BPR loss is 4.12E+01. Sample per second: 7467
SLIM_S_BPR_Recommender: Epoch 723 of 750. Elapsed time 15.12 min
Processed 13984 (100.0%) in 2.52 sec. BPR loss is 3.88E+01. Sample per second: 5556
SLIM_S_BPR_Recommender: Epoch 724 of 750. Elapsed time 15.15 min
Processed 13984 (100.0%) in 2.20 sec. BPR loss is 3.98E+01. Sample per second: 6350
SLIM_S_BPR_Recommender: Epoch 725 of 750. Elapsed time 15.17 min
Processed 13984 (100.0%) in 1.88 sec. BPR loss is 3.90E+01. Sample per second: 7421
SLIM_S_BPR_Recommender

In [15]:
import pandas as pd

user_test_path = '../data/data_target_users_test.csv'
user_test_dataframe = pd.read_csv(filepath_or_buffer=user_test_path,
								sep=",",
								dtype={0:int})

subm_set = user_test_dataframe.to_numpy().T[0]


subm_res = {"user_id":[], "item_list":[]}

for user_id in subm_set:
	subm_res["user_id"].append(user_id)
	res = recommender.recommend(user_id, K)
	res = ' '.join(map(str, res))
	if user_id < 3:
		print(user_id)
		print(res)
	subm_res["item_list"].append(res)


# print(subm_res)

submission = pd.DataFrame.from_dict(subm_res)
# submission

from datetime import datetime
now = datetime.now() # current date and time


submission.to_csv('../subs/submission {:%Y_%m_%d %H_%M_%S}SLIMSUBRAND.csv'.format(now), index=False)

0
16359 5433 640 6061 7143 12752 9130 3650 2243 1792
1
3473 940 14358 16365 640 10118 16012 14400 2118 3829
2
6177 1551 14181 13963 17584 16221 15119 16846 17330 5324
